Here we will compare different samplig methods for the target `donut_target`, specifically the ess method 

### Importing the needed libraries 

In [1]:
from cuqi.distribution import DistributionGallery, Gaussian, JointDistribution
from cuqi.testproblem import Poisson1D
from cuqi.problem import BayesianProblem
import cuqi
import inspect
import numpy as np
import matplotlib.pyplot as plt
from cuqi.sampler import MH, CWMH, ULA, MALA, NUTS
import time
import scipy.stats as sps
from scipy.stats import gaussian_kde
import pandas as pd
import cProfile, pstats, io
from pstats import SortKey
import TableAutomization as TA

### The donut distribution


Out of all of the sampling methods we choose 5:
- MH with fixed samlpler
- MH with adapted sampler
- ULA
- MALA 
- NUTSA

We create a general sampling function, that given certain inputs will compute the according MCMC sampling

In [2]:
# The donut distribution 
target_donut = DistributionGallery("donut")


In [3]:
TA.create_table(target_donut, 0.05, 100,2, x0 = np.array([0,0]), seed = 12 )

Sample 102 / 102

Average acceptance rate: 0.61 

Sample 102 / 102

Average acceptance rate: 0.55 MCMC scale: 0.2801305356910719 

Sample 102 / 102
Sample 102 / 102
Sample 102 / 102


,Sampling Method,No. of Samples,No. of Burn-ins,Scaling Factor,ESS (v0),ESS (v1),LogPDF
0,MH_fixed,100,2,0.05,1.388,7.172,102.0
1,MH_adapted,100,2,0.05,1.958,6.839,102.0
2,ULA,100,2,0.05,2.435,23.095,102.0
3,MALA,100,2,0.05,100.000,100.000,102.0
4,NUTS,100,2,0.05,33.394,3.729,0.0
